In [1]:
!pip install -q transformers datasets accelerate

In [2]:
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer

In [3]:
ds = load_dataset("databricks/databricks-dolly-15k")
ds["train"] = ds["train"].shuffle(seed=42).select(range(2000))

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [4]:
def format_prompt(example):
    prompt = f"""### Instruction:
{example['instruction']}

### Context:
{example['context']}

### Response:
{example['response']}"""
    return {"text": prompt}

train_ds = ds["train"].map(format_prompt)

In [5]:
model_name = "EleutherAI/pythia-410m"

tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    torch_dtype=None
)

Loading weights:   0%|          | 0/292 [00:00<?, ?it/s]

In [6]:
def tokenize(batch):
    tokens = tokenizer(
        batch["text"],
        truncation=True,
        padding="max_length",
        max_length=256
    )
    tokens["labels"] = tokens["input_ids"].copy()
    return tokens

tokenized_ds = train_ds.map(
    tokenize,
    batched=True,
    remove_columns=train_ds.column_names
)

In [7]:
training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=8,
    num_train_epochs=1,
    logging_steps=20,
    save_strategy="no",
    report_to="none"
)

In [8]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_ds
)

trainer.train()

Step,Training Loss
20,514.612793
40,0.000000
60,0.000000
80,0.000000
100,0.000000
120,0.000000
140,0.000000
160,0.000000
180,0.000000
200,0.000000


TrainOutput(global_step=250, training_loss=41.1690234375, metrics={'train_runtime': 78.5608, 'train_samples_per_second': 25.458, 'train_steps_per_second': 3.182, 'total_flos': 1086943395840000.0, 'train_loss': 41.1690234375, 'epoch': 1.0})